In [6]:
import pandas as pd
import numpy as np
import ensemble_boxes 
from pycocotools.coco import COCO
import itertools
import os
from tqdm import tqdm

In [13]:
indices = [0, 1, 2, 3, 4]
sizes = [720, 720, 680, 720, 720]

algorithms = ['nms', 'soft_nms', 'non_maximum_weighted', 'weighted_boxes_fusion']
iou_thresholds = np.arange(0.1, 0.91, 0.1).tolist()
output_name_format = '1013_deta_{num_file}_fold_{algorithm}_{iou_threshold:.1f}.csv'
output_fold = 'result'

# submission_file_format = '../../result/jozhang97/deta-swin-large_{i}_img_size_{size}/deta-swin-large_{i}_img_size_{size}.csv'
annotation_path = '/data/ephemeral/home/Dongjin/level2-objectdetection-cv-07/Split_data/valid_3_5.json'
# submission_file_paths = []

# for i, size in zip(indices, sizes):
#     submission_file_path = submission_file_format.format(i=i, size=size)
#     submission_file_paths.append(submission_file_path)

submission_file_paths = ['result/deta-swin-large_3_img_size_720_valid.csv',
                         'result/deta-swin-large_3_img_size_720_valid_hFlip.csv']

submission_df = [pd.read_csv(file) for file in submission_file_paths]
image_ids = submission_df[0]['image_id'].tolist()

coco = COCO(annotation_path)

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


In [14]:
def get_box_score_label(submission_df, image_id, image_info):
    boxes_list = []
    scores_list = []
    labels_list = []

    # 각 submission file 별로 prediction box좌표 불러오기
    for df in submission_df:
        predict_string = df[df['image_id'] == image_id]['PredictionString'].tolist()[0]
        predict_list = str(predict_string).split()

        if len(predict_list)==0 or len(predict_list)==1:
            continue

        predict_list = np.reshape(predict_list, (-1, 6))
        box_list = []

        for box in predict_list[:, 2:6].tolist():
            # box의 각 좌표를 float형으로 변환한 후 image의 넓이와 높이로 각각 정규화
            image_width = image_info['width']
            image_height = image_info['height']

            box[0] = float(box[0]) / image_width
            box[1] = float(box[1]) / image_height
            box[2] = float(box[2]) / image_width
            box[3] = float(box[3]) / image_height

            box = np.clip(box, 0, 1).tolist()
            box_list.append(box)

        boxes_list.append(box_list)
        scores_list.append(list(map(float, predict_list[:, 1].tolist())))
        labels_list.append(list(map(int, predict_list[:, 0].tolist())))

    return boxes_list, scores_list, labels_list


def get_prediction(boxes, scores, labels, image_width, image_height):
    prediction_string = ""
    for box, score, label in zip(boxes, scores, labels):
        xmin, ymin = box[0] * image_width, box[1] * image_height
        xmax, ymax = box[2] * image_width, box[3] * image_height
        prediction_string += f'{label:.0f} {score:.5f} {xmin:.2f} {ymin:.2f} {xmax:.2f} {ymax:.2f} '
    return prediction_string

In [32]:
coco.getImgIds()

[56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 81,
 117,
 118,
 121,
 122,
 123,
 133,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 175,
 176,
 177,
 178,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 228,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 319,
 320,
 361,
 362,
 363,
 364,
 365,
 366,
 367,
 383,
 426,
 427,
 433,
 434,
 435,
 436,
 437,
 438,
 439,
 470,
 471,
 532,
 542,
 543,
 544,
 545,
 546,
 547,
 548,
 549,
 582,
 612,
 613,
 630,
 688,
 711,
 712,
 713,
 714,
 715,
 716,
 717,
 742,
 743,
 744,
 745,
 746,
 747,
 748,
 753,
 786,
 787,
 788,
 810,
 811,
 812,
 845,
 846,
 847,
 848,
 849,
 850,
 851,
 852,
 853,
 854,
 855,
 856,
 857,
 858,
 859,
 860,
 861,
 862,
 863,
 864,
 865,
 936,
 949,
 950,
 951,
 952,
 994,
 995,
 996,
 997,
 998,
 999,
 1008,
 1021,
 1022,
 1023,
 1024,
 1025,
 10

In [15]:
conf = list(enumerate(list(itertools.product(algorithms, iou_thresholds)))) # ensemble할 조건 리스트
results = [{'prediction_strings': [], 'file_names': []} for i in range(len(conf))] # 결과 저장 리스트 선언

# 각 image id 별로 submission file에서 box좌표 추출
for i, image_id in enumerate(tqdm(image_ids)):    
    prediction_string = ''
    boxes_list = []
    scores_list = []
    labels_list = []

    image_info = coco.loadImgs(int(image_id[6:-4]))[0]
    image_width = image_info['width']
    image_height = image_info['height']

    boxes_list, scores_list, labels_list = get_box_score_label(submission_df, image_id, image_info)
    
    # 예측 box가 있다면 ensemble 수행
    if len(boxes_list):
        # ensemble에 필요한 인자: [box의 lists, confidence score의 lists, label의 list, iou에 사용할 threshold]
        for j, (algorithm, iou_threshold) in conf:
            boxes, scores, labels = getattr(ensemble_boxes, algorithm)(boxes_list, scores_list, labels_list, iou_thr=iou_threshold)
            prediction_string = get_prediction(boxes, scores, labels, image_width, image_height)

            results[j]['prediction_strings'].append(prediction_string)
            results[j]['file_names'].append(image_id)    

  0%|          | 0/958 [00:00<?, ?it/s]


KeyError: 0

In [7]:
# 결과 저장하기
num_file = len(submission_file_paths)
os.makedirs(output_fold, exist_ok=True)

for i, (algorithm, iou_threshold) in conf:
    output_name = output_name_format.format(num_file=num_file, algorithm=algorithm, iou_threshold=iou_threshold)
    output_path = os.path.join(output_fold, output_name)

    submission = pd.DataFrame()
    submission['PredictionString'] = results[i]['prediction_strings']
    submission['image_id'] = results[i]['file_names']

    submission.to_csv(output_path, index=False)
    submission.head()

56